<a href="https://colab.research.google.com/github/renataeugenia/ChallengeAlura_DataScience/blob/main/Notebook/Challenge_Alura_DataScience_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Alura Data Science - 2º fase 19/12/2022

In [26]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [27]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [28]:
import findspark
findspark.init()

In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Challenge Alura fase_2") \
    .getOrCreate()

In [30]:
spark

In [31]:
%%time
dataset_parquet = spark.read.parquet(
    '/content/drive/MyDrive/Challenge_Alura/semana-2/dataset_transformado_parquet'
)

CPU times: user 2.69 ms, sys: 0 ns, total: 2.69 ms
Wall time: 195 ms


In [32]:
dataset_parquet.show(5, truncate=False)

+------------------------------------+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+------------+------------+----------+----+-----+------+
|id                                  |andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro      |zona        |condominio|iptu|tipo |valor |
+------------------------------------+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4eff-891d-f8a16efc1989|0    |43   

In [33]:
dataset_parquet.count()

66562

**card: Seleção de features** 
* ( Analise as colunas area_total e area_util) 

In [34]:
dataset_parquet.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [35]:
dataset_parquet\
    .select('area_total', 'area_util')\
    .groupby('area_total', 'area_util')\
    .count()\
    .show()

+----------+---------+-----+
|area_total|area_util|count|
+----------+---------+-----+
|       217|      217|   30|
|       360|      360|   45|
|       600|      600|    8|
|        50|       46|    7|
|       299|      299|   13|
|        72|       66|    2|
|       130|      137|    1|
|       240|      217|    1|
|       221|      201|    1|
|       447|      447|    1|
|       101|      100|    1|
|       165|      150|    1|
|      1100|     1100|    1|
|       397|      397|    1|
|      null|      296|    3|
|        70|       69|    4|
|       337|      337|   12|
|       319|      319|   11|
|      null|      125|   40|
|      null|      800|    1|
+----------+---------+-----+
only showing top 20 rows



*Essas colunas parecem ter muitos valores iguais (conferir isso é importante). Será que é necessário deixar as duas colunas no nosso DataFrame? Qual delas possui mais dados nulos?*

In [38]:
dataset_parquet = dataset_parquet\
    .drop('area_total')

In [39]:
dataset_parquet.show(5, truncate=False)

+------------------------------------+-----+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+------------+------------+----------+----+-----+------+
|id                                  |andar|area_util|banheiros|caracteristicas                                                                                                                |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro      |zona        |condominio|iptu|tipo |valor |
+------------------------------------+-----+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4eff-891d-f8a16efc1989|0    |43       |1        |[Churrasqueira, Ac

**card: Converter os tipos das colunas**
* "andar", "banheiros", "suites" e "quartos" para o tipo inteiro;
* "area_util", "condominio", "iptu" e "valor" para o tipo double.

In [40]:
dataset_parquet.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [41]:
from pyspark.sql.types import IntegerType, DoubleType

In [43]:
dataset_parquet\
    .withColumn('andar', dataset_parquet['andar'].cast(IntegerType()))\
    .withColumn('banheiros', dataset_parquet['banheiros'].cast(IntegerType()))\
    .withColumn('quartos', dataset_parquet['quartos'].cast(IntegerType()))\
    .withColumn('suites', dataset_parquet['suites'].cast(IntegerType()))\
    .withColumn('vaga', dataset_parquet['vaga'].cast(IntegerType()))\
    .withColumn('area_util', dataset_parquet['area_util'].cast(DoubleType()))\
    .withColumn('condominio', dataset_parquet['condominio'].cast(DoubleType()))\
    .withColumn('iptu', dataset_parquet['iptu'].cast(DoubleType()))\
    .withColumn('valor', dataset_parquet['valor'].cast(DoubleType()))\
    .printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



In [46]:
dataset_parquet.limit(10).toPandas()

,id,andar,area_util,banheiros,caracteristicas,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,bairro,zona,condominio,iptu,tipo,valor
0,03a386b6-7ab8-4eff-891d-f8a16efc1989,0,43,1,"[Churrasqueira, Academia, Playground, Salão de...",2,NaN,Usado,Apartamento,Residencial,1,Realengo,Zona Oeste,285,None,Venda,22999
1,1fe78d41-b8e0-4d21-a1a8-72cb7863f441,0,44,1,[],2,0.0,Usado,Apartamento,Residencial,0,Irajá,Zona Norte,170,0,Venda,110000
2,1fa1c1e5-e98c-433f-9675-1b43d7d6da01,4,55,1,[],2,0.0,Usado,Apartamento,Residencial,1,Cosmos,Zona Oeste,None,None,Venda,115000
3,a6ab01ae-3d40-40e7-bb6a-f2334e0abbe0,2,55,1,[],2,0.0,Usado,Apartamento,Residencial,0,Tomás Coelho,Zona Norte,None,None,Venda,115000
4,1d8ff7a2-16bd-4e3f-a92e-1b85f923ebb0,0,50,1,[],1,NaN,Usado,Apartamento,Residencial,0,Centro,Zona Central,1300,2679,Venda,125000
5,40a2cbde-946b-42da-b7f2-b80057c7a47f,2,47,1,"[Condomínio fechado, Portaria 24h, Salão de fe...",2,0.0,Usado,Apartamento,Residencial,1,Campo Grande,Zona Oeste,380,None,Venda,138000
6,be269ddf-a4a3-4a72-bd54-93cdccf512dd,0,45,1,"[Churrasqueira, Condomínio fechado, Piscina]",2,0.0,Usado,Apartamento,Residencial,1,Campo Grande,Zona Oeste,356,0,Venda,140000
7,4b7e3d59-34fd-4bac-9232-1015cb3b3a24,3,48,1,"[Churrasqueira, Piscina, Playground, Salão de ...",2,NaN,Usado,Apartamento,Residencial,1,Campo Grande,Zona Oeste,0,0,Venda,150000
8,a1a0c2ac-4fa8-4b02-a087-f9d76b6bccb7,9,55,1,"[Churrasqueira, Elevador, Salão de festas, Con...",2,NaN,Usado,Apartamento,Residencial,1,Freguesia (Jacarepaguá),Zona Oeste,780,660,Venda,160000
9,390d2b88-e06d-47e5-ba7f-a680130ac5f0,0,70,2,[Salão de festas],2,0.0,Usado,Apartamento,Residencial,1,Praça Seca,Zona Oeste,420000,None,Venda,159000
